In [1]:
import h5py
import matplotlib.pyplot as plt
import torch
import os
import numpy as np


In [2]:
data_dir = '/home/qutrll/data/pot_pick_place/'
episodes = os.listdir(data_dir)
num_episodes = len(episodes)
split = 0.9
train_episodes = int(num_episodes*split)
val_episodes = num_episodes - train_episodes
print('Number of episodes: ', num_episodes)
print('Number of training episodes: ', train_episodes)
print('Number of validation episodes: ', val_episodes)


Number of episodes:  55
Number of training episodes:  49
Number of validation episodes:  6


In [3]:
train_indices = np.random.choice(num_episodes, size=train_episodes, replace=False)
val_indices = np.array([i for i in range(num_episodes) if i not in train_indices])
print('Train indices: ', train_indices)
print('Val indices: ', val_indices)


Train indices:  [12  1 20  2 51 17 21 41 54 28 29 18 32 27  7 37 15 48 30 10 39 50 43 23
  9  4 13 14  3 33 11 53  8 45  5 22 38 26 24 25 16 34 31 47  0 52 46 36
 42]
Val indices:  [ 6 19 35 40 44 49]


In [4]:
B = 32
T = 1
keys = ['action', 'q_pos', 'oh_images', 'wrist_images', 'field_images', 'progress']
episodes = sorted(episodes)
episodes = [episodes[i] for i in train_indices]
print(f"Loading episodes: {episodes}")
episode_files = [h5py.File(data_dir + episode, 'r') for episode in episodes]
num_keys = len(episode_files[0].keys())


Loading episodes: ['2.hdf5', '1.hdf5', '27.hdf5', '10.hdf5', '6.hdf5', '24.hdf5', '28.hdf5', '47.hdf5', '9.hdf5', '35.hdf5', '36.hdf5', '25.hdf5', '39.hdf5', '34.hdf5', '15.hdf5', '43.hdf5', '22.hdf5', '53.hdf5', '37.hdf5', '18.hdf5', '45.hdf5', '55.hdf5', '49.hdf5', '30.hdf5', '17.hdf5', '12.hdf5', '20.hdf5', '21.hdf5', '11.hdf5', '4.hdf5', '19.hdf5', '8.hdf5', '16.hdf5', '50.hdf5', '13.hdf5', '3.hdf5', '44.hdf5', '33.hdf5', '31.hdf5', '32.hdf5', '23.hdf5', '40.hdf5', '38.hdf5', '52.hdf5', '0.hdf5', '7.hdf5', '51.hdf5', '42.hdf5', '48.hdf5']


In [5]:
def extract_data_from_hdf5(file, keys):
    data = []
    for key in keys:
        if 'image' in key:
            d = torch.from_numpy(file[key][:]).float() / 255.0
            d = d.permute(0, 3, 1, 2) 
        else:
            d = torch.from_numpy(file[key][:]).float()
        data.append(d)
    return data           


In [6]:
# Initialize an empty list to store data from each file
action = []
q_pos = []
oh_images = []
wrist_images = []
field_images = []
progress = []
for file in episode_files:
    data = extract_data_from_hdf5(file, keys)
    action.append(data[0])
    q_pos.append(data[1])
    oh_images.append(data[2])
    wrist_images.append(data[3])
    field_images.append(data[4])
    progress.append(data[5])


In [1]:
from dataloader import DataLoaderLite
import numpy as np
import os

# Set random seed for reproducibility
np.random.seed(42)

# Parameters
B = 32  # Batch size
T = 1   # Sequence length
num_episodes = 32  # Total number of episodes
split = "train"
data_dir = "/home/qutrll/data/pot_pick_place/"

# Generate random indices
indices = np.random.choice(num_episodes, size=B, replace=False)

# Create DataLoaderLite instance
dataloader = DataLoaderLite(data_dir, B, T, split, indices)


Loading train episodes: ['48.hdf5', '12.hdf5', '45.hdf5', '27.hdf5', '52.hdf5', '39.hdf5', '37.hdf5', '6.hdf5', '47.hdf5', '34.hdf5', '17.hdf5', '31.hdf5', '5.hdf5', '0.hdf5', '36.hdf5', '41.hdf5', '30.hdf5', '46.hdf5', '14.hdf5', '43.hdf5', '51.hdf5', '1.hdf5', '19.hdf5', '25.hdf5', '10.hdf5', '55.hdf5', '44.hdf5', '2.hdf5', '32.hdf5', '11.hdf5', '22.hdf5', '18.hdf5']
Found 32 episodes


In [2]:
# Get a batch of data
actions, q_pos, images, progress = dataloader.next_batch()

# Print shapes of returned tensors
print(f"Actions shape: {actions.shape}")
print(f"Q_pos shape: {q_pos.shape}")
print(f"Images shape: {images.shape}")
print(f"Progress shape: {progress.shape}")


Actions shape: torch.Size([32, 7])
Q_pos shape: torch.Size([32, 7])
Images shape: torch.Size([32, 3, 3, 240, 424])
Progress shape: torch.Size([32, 1])


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
actions = actions.to(device)
q_pos = q_pos.to(device)
images = images.to(device)
progress = progress.to(device)


In [4]:
from model import PolicyCNNMLP

policy = PolicyCNNMLP().to(device)


In [5]:
action = policy(images, q_pos, progress)


In [6]:
action.shape


torch.Size([32, 8])

In [7]:
# Iterate 100 times
import time
start = time.time()
for i in range(100):
    action = policy(images, q_pos, progress)
end = time.time()
print(f"Time taken: {end - start} seconds")
print(f"Average time per forward pass: {(end - start) / 100} seconds")


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 23.55 GiB of which 187.19 MiB is free. Including non-PyTorch memory, this process has 23.35 GiB memory in use. Of the allocated memory 22.81 GiB is allocated by PyTorch, and 91.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)